In [9]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_react.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"
import django
django.setup()

In [10]:
from instagram.models import Post

In [13]:
qs = Post.objects.all().order_by('-id')[:10]
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" ORDER BY "instagram_post"."id" DESC LIMIT 10


<QuerySet [<Post: 다섯번째 메세지>, <Post: 두번째 메세지>, <Post: 첫번째 메세지>]>

In [14]:
for post in qs:
    print('id: {id}, message: {message} ({created_at})'.format(**post.__dict__))

id: 5, message: 다섯번째 메세지 (2023-05-09 11:21:59.931791+00:00)
id: 4, message: 두번째 메세지 (2023-05-09 11:21:52.433578+00:00)
id: 1, message: 첫번째 메세지 (2023-05-09 11:21:00.651094+00:00)


In [21]:
query = '메세지' # 검색어

qs = Post.objects.all().filter(message__icontains=query).order_by('id')
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE "instagram_post"."message" LIKE %메세지% ESCAPE '\' ORDER BY "instagram_post"."id" ASC


<QuerySet [<Post: 첫번째 메세지>, <Post: 두번째 메세지>, <Post: 다섯번째 메세지>]>

In [22]:
qs = Post.objects.all()
qs = qs.exclude(message__icontains=query)
qs = qs.order_by('-id')
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE NOT ("instagram_post"."message" LIKE %메세지% ESCAPE '\') ORDER BY "instagram_post"."id" DESC


<QuerySet []>

In [26]:
qs = Post.objects.all()
qs.get(id__lte=10)

MultipleObjectsReturned: get() returned more than one Post -- it returned 3!

In [27]:
qs.first()

<Post: 첫번째 메세지>

In [29]:
qs.last()

<Post: 다섯번째 메세지>

In [30]:
print(qs.none().first())

None


In [31]:
from django.db.models import Q

In [43]:
qs = Post.objects.all()
# qs = qs.filter(id__gte = 2, message__icontains=query)
# 비트연산자 사용 가능 & : and | : or
qs = qs.filter(Q(id__gte = 2) & Q(message__icontains=query))

# 다음과 같이도 활용 가능
cond = Q(id__gte=2) | Q(message__icontains=query)
# cond &= Q(...)
qs = qs.filter(cond)
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE ("instagram_post"."id" >= 2 AND "instagram_post"."message" LIKE %메세지% ESCAPE '\' AND ("instagram_post"."id" >= 2 OR "instagram_post"."message" LIKE %메세지% ESCAPE '\'))
